In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

batchsize=1

class DCGAN_D(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ndf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        main.add_module('final_1024.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, 1024, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
            output = self.main(input)
            
        return output

class DCGAN_G(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ngf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output 
###############################################################################

IndentationError: unindent does not match any outer indentation level (<ipython-input-2-bdb6f3c237ca>, line 65)

In [ ]:
import cv2
import numpy as np
batchSize = 1

def bonemarrow_cell():
    X = np.load("../data/data.npy")
    img = X
    X = np.asarray([cv2.resize(x, (32,32)) for x in X])
    X = np.asarray([x[:,:,::-1].transpose((2,0,1)) for x in X])
    X = X.astype(np.float32)/(255.0/2) - 1.0
    return X

X_train = bonemarrow_cell()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

In [ ]:
dict_G = torch.load('/disk1/cell_work/models/0626/netG_256_epoch_%d.pth' % (4153))
dict_D = torch.load('/disk1/cell_work/models/0626/netD_256_epoch_%d.pth' % (4153))
del(dict_D['main.final.256-1.conv.weight'])

In [ ]:
feature_dict = {}
netD = DCGAN_D()
netD.load_state_dict(dict_D)
print(netD)
data_iter = iter(dataloader)
for i,data in enumerate(data_iter):
    feature_dict[i] = netD(Variable(data)).data.numpy().reshape((1,1024))
    
feature = np.zeros((max(feature_dict.keys()),1024),dtype=np.float32)
for i,data in enumerate(feature):
    data[:] = feature_dict[i] 

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit_predict(feature)
index_0,index_1 = np.where(kmeans==0)[0],np.where(kmeans==1)[0]

cluster3_0 = np.zeros((index_0.shape[0],3,32,32),dtype=np.float32)
cluster3_1 = np.zeros((index_1.shape[0],3,32,32),dtype=np.float32)
cluster3_2 = np.zeros((index_2.shape[0],3,32,32),dtype=np.float32)
data_iter = iter(dataloader)
m=n=l=0
for i,data in enumerate(data_iter):
    if i in index_0:
        cluster_0[m] = data[0].numpy()
        m+=1
    if i in index_1:
        cluster_1[n] = data[0].numpy()
        n+=1
    if i in index_1:
        cluster_1[l] = data[0].numpy()
        l+=1
vutils.save_image(torch.from_numpy(cluster_0), 'cluster3_0.png',normalize=True)
vutils.save_image(torch.from_numpy(cluster_1), 'cluster3_1.png',normalize=True)
vutils.save_image(torch.from_numpy(cluster_2), 'cluster3_2.png',normalize=True)